In [37]:
from pyspark.sql import SparkSession
import pyspark.pandas as ps
spark = SparkSession\
    .builder\
    .appName("MergeDataset")\
    .getOrCreate()

In [51]:
fbref_MatchStats = ps.read_csv('./data/fbref_MatchStats.csv')
fbref_MatchStats['fbrefMatchId'] = fbref_MatchStats['fbrefMatchId'].astype(str)

c:\Users\super\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [52]:
def Fbref_MergeHomeAwayMatchStats(fbref_MatchStats):
    fbref_MatchStats.rename(
        columns={
            'touches44':'touches',
            'Touches21':'Touches',
        }
    )
    home_MatchStats = fbref_MatchStats.loc[fbref_MatchStats['is_home_team'] == 'Yes']
    away_MatchStats = fbref_MatchStats.loc[fbref_MatchStats['is_home_team'] == 'No']
    home_columns = ['home_' + column for column in home_MatchStats.columns.tolist()]
    away_columns = ['away_' + column for column in away_MatchStats.columns.tolist()]
    home_MatchStats.columns = home_columns
    away_MatchStats.columns = away_columns
    home_MatchStats['home_fbrefMatchId'] = home_MatchStats['home_fbrefMatchId'].astype(str)
    away_MatchStats['away_fbrefMatchId'] = away_MatchStats['away_fbrefMatchId'].astype(str)
    merged_MatchStats = ps.merge(home_MatchStats, away_MatchStats, left_on='home_fbrefMatchId',right_on = 'away_fbrefMatchId', how = 'inner')
    
    return merged_MatchStats

In [53]:
merged_MatchStats = Fbref_MergeHomeAwayMatchStats(fbref_MatchStats)
merged_MatchStats.head(2)
merged_MatchStats.to_pandas().to_csv('./data/fbref_MergedMatchStats.csv',index=False,encoding='utf-8-sig')
    

c:\Users\super\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [ ]:
# spark.stop()